This notebook creates features based on the lab/vital measurements and clinical orders datasets.

In [1]:
import pandas as pd
import numpy as np

In [239]:
final = pd.read_csv('semi_final2.csv').drop(columns = ['Unnamed: 0'])

print(final.shape)
print(final.head())

(12509, 32)
  ENCOUNTER_NUM  PATIENT_DK          FROM_SERVICE  \
0  0013da1c712c      668421  TMC GENERAL MEDICINE   
1  0014899413c8       69698  TMA GENERAL MEDICINE   
2  0016fd45df95      838638  TMC GENERAL MEDICINE   
3  0063dea6173d      127708  TMC GENERAL MEDICINE   
4  00b661c2b87e      850511  TME GENERAL MEDICINE   

                       ADT_DISCHARGE  \
0              Home Without Services   
1              Home Without Services   
2  Long Term Care/Nursing Home/Detox   
3              Home Without Services   
4              Home Without Services   

                              MRP_DIAGNOSIS        age  gim_to_outcome  \
0  Cutan abscess furuncle & carbuncle trunk  41.869648        2.674120   
1                                       NaN  89.221340        3.710741   
2                  Congestive heart failure  91.860508       24.363495   
3                                       NaN  61.269224        4.557257   
4                                       NaN  71.207212    

# Labs

In [3]:
test = pd.read_csv('outcsv_test_numeric_data.csv')
train = pd.read_csv('outcsv_train_train_numeric_variables.csv')
val = pd.read_csv('outcsv_valid_valid_numeric_variables.csv')

labs = pd.concat([train, test, val], axis = 0)
labs = labs.sort_values(by = ['ENCOUNTER_NUM', 'variable', 'time']).reset_index(drop = True)

print(labs.head())
print(labs.shape)

  ENCOUNTER_NUM            variable       time  numeric_value
0  00044d6e3352  alcohol_sciwascore   4.728611            7.0
1  00044d6e3352  alcohol_sciwascore  92.728611            9.0
2  00044d6e3352    in_out_acatheter   6.261944         6000.0
3  00044d6e3352    in_out_acatheter  29.678611        14500.0
4  00044d6e3352    in_out_acatheter  42.145278         7500.0
(5985982, 4)


In [4]:
testb = pd.read_csv('outcsv_test_baseline_values.csv')
trainb = pd.read_csv('outcsv_train_train_baseline_values.csv')
valb = pd.read_csv('outcsv_valid_valid_baseline_values.csv')

labsb = pd.concat([trainb, testb, valb], axis = 0)
labsb.head()

,ENCOUNTER_NUM,lab_co2,lab_cl,lab_na,lab_agap,lab_hct,lab_mch,lab_mchc,lab_rbc,lab_iwbcr,...,lab_vbe,lab_vtco2,lab_vpco2,lab_vph,vital_spulse,vital_sbpsystolic,vital_sbpdiastolic,vital_srespirations,vital_so2saturation,vital_stemperature
0,645b4b89e25a,0.176630,0.221591,0.071121,0.456731,0.560886,0.690549,0.877119,0.524096,0.515942,...,0.069914,0.231985,0.626866,0.0,0.394366,0.397959,0.444444,0.230769,0.681818,0.4
1,47ffd77e2a26,0.014493,0.764310,0.444444,0.521368,0.850923,0.510976,0.725424,0.881928,0.675365,...,0.000000,0.000000,0.014925,0.0,0.394366,0.397959,0.444444,0.230769,0.681818,0.4
2,9374ae8ddcbb,0.304348,0.554113,0.487685,0.417582,0.845756,0.703659,0.708475,0.765663,0.564667,...,0.000000,0.231985,0.850746,0.0,0.394366,0.397959,0.444444,0.230769,0.681818,0.4
3,03996793b1a4,0.341304,0.642424,0.767241,0.546154,0.168656,0.395265,0.643071,0.281184,0.558177,...,0.000000,0.000000,0.149254,0.0,0.394366,0.397959,0.444444,0.230769,0.681818,0.4
4,8aad2e604037,0.086957,0.607955,0.478448,0.859729,0.538745,0.507114,0.870056,0.613956,0.471388,...,0.000000,0.000000,0.000000,0.0,0.394366,0.397959,0.444444,0.230769,0.681818,0.4


In [5]:
labsb = labsb.melt(id_vars = 'ENCOUNTER_NUM')

labsb.loc[:, 'time'] = 0
labsb.rename(columns = {'value': 'numeric_value'}, inplace = True)

labsb.head()

,ENCOUNTER_NUM,variable,numeric_value,time
0,645b4b89e25a,lab_co2,0.176630,0
1,47ffd77e2a26,lab_co2,0.014493,0
2,9374ae8ddcbb,lab_co2,0.304348,0
3,03996793b1a4,lab_co2,0.341304,0
4,8aad2e604037,lab_co2,0.086957,0


In [6]:
(labs.numeric_value <= 0).sum()

521941

In [7]:
(labsb.numeric_value <= 0).sum()

32711

In [8]:
labs_all = pd.concat([labsb[labs.columns], labs], axis = 0)
labs_all.loc[labs_all.numeric_value <= 0, 'numeric_value'] = np.nan

labs_all.head()

,ENCOUNTER_NUM,variable,time,numeric_value
0,645b4b89e25a,lab_co2,0.0,0.176630
1,47ffd77e2a26,lab_co2,0.0,0.014493
2,9374ae8ddcbb,lab_co2,0.0,0.304348
3,03996793b1a4,lab_co2,0.0,0.341304
4,8aad2e604037,lab_co2,0.0,0.086957


In [9]:
labs_all.numeric_value.isnull().sum()

554652

In [10]:
keep_types = ['alcohol_sciwascore', 'diabetic_spocglucresult', 'in_out_acatheter', 'in_out_atmsincontinent',
              'in_out_siv23and13', 'in_out_sivnormalsaline', 'lab_acet', 'lab_alact', 'lab_alb', 'lab_alp',
              'lab_alt', 'lab_amy', 'lab_apco2', 'lab_aph', 'lab_apo2', 'lab_asa', 'lab_ast', 'lab_atco2', 
              'lab_b12', 'lab_bc', 'lab_bnps', 'lab_ca', 'lab_ck', 'lab_cl', 'lab_co2', 'lab_cr', 'lab_crp', 
              'lab_etoh', 'lab_fer', 'lab_glur', 'lab_hba1', 'lab_hgb', 'lab_iwbcr', 'lab_k', 'lab_lip', 
              'lab_mg', 'lab_na', 'lab_ph', 'lab_plt', 'lab_po4', 'lab_rinr', 'lab_rpt', 'lab_rptt', 'lab_sat', 
              'lab_tbil', 'lab_tni', 'lab_tpr', 'lab_tsh', 'lab_urea', 'lab_vlact', 'lab_vpco2', 'lab_vph',
              'lab_vpo2', 'lab_vtco2', 'shift_assess_scvhrtrate', 'shift_assess_srpfio2b', 'shift_assess_srpo2lmin',
              'skin_abradenscore', 'vital_sbpdiastolic', 'vital_sbpsystolic', 'vital_sfio2', 'vital_so2saturation',
              'vital_spainintmove', 'vital_spainintrest', 'vital_spulse', 'vital_srespirations', 'vital_stemperature']

labs_all = labs_all[(labs_all.variable.isin(keep_types)) & (labs_all.ENCOUNTER_NUM.isin(final.ENCOUNTER_NUM))]
labs_all = labs_all.sort_values(by = ['ENCOUNTER_NUM', 'variable', 'time']).reset_index(drop = True)

In [11]:
labs_all.numeric_value.isnull().sum()

306918

In [188]:
labs_numorders = labs_all[labs_all.time > 0].groupby(['ENCOUNTER_NUM'])['variable'].count().rename('labs_num')
labs_numtypes = labs_all[labs_all.time > 0].groupby(['ENCOUNTER_NUM'])['variable'].nunique().rename('labs_numtypes')

In [12]:
# Impute invalid measurements with the prior value
prevs = labs_all.groupby(['ENCOUNTER_NUM', 'variable']).apply(lambda x: x.sort_values(by = 'time').numeric_value.shift())

prevs

ENCOUNTER_NUM  variable                       
00044d6e3352   alcohol_sciwascore      0                   NaN
                                       1              7.000000
               in_out_acatheter        2                   NaN
                                       3           6000.000000
                                       4          14500.000000
                                       5           7500.000000
               in_out_siv23and13       6                   NaN
                                       7          15000.000000
               in_out_sivnormalsaline  8                   NaN
                                       9           3750.000000
               lab_alb                 10                  NaN
                                       11             0.793103
               lab_alp                 12                  NaN
                                       13             0.032653
               lab_alt                 14                  NaN
        

In [13]:
labs_all.loc[:, 'numeric_value_prev'] = prevs.reset_index(drop = True)

labs_all.loc[:, 'numeric_value_imputed'] = list(map(lambda x, y: y if np.isnan(x) else x, 
                                                    labs_all.numeric_value, labs_all.numeric_value_prev))

In [14]:
print(sum(labs_all.numeric_value <= 0))
print(labs_all.numeric_value.isnull().sum())

0
306918


In [15]:
print(sum(labs_all.numeric_value_imputed <= 0))
print(labs_all.numeric_value_imputed.isnull().sum())

0
282500


In [16]:
sum(labs_all.numeric_value != labs_all.numeric_value_imputed)

306918

In [17]:
labs_sub = labs_all.groupby(['ENCOUNTER_NUM', 'variable']).apply(lambda x: x.sort_values(by = 'time').tail(1)).reset_index(drop = True)

labs_sub

,ENCOUNTER_NUM,variable,time,numeric_value,numeric_value_prev,numeric_value_imputed
0,00044d6e3352,alcohol_sciwascore,92.728611,9.000000,7.000000,9.000000
1,00044d6e3352,in_out_acatheter,54.078611,8500.000000,7500.000000,8500.000000
2,00044d6e3352,in_out_siv23and13,54.078611,6250.000000,15000.000000,6250.000000
3,00044d6e3352,in_out_sivnormalsaline,42.145278,15000.000000,3750.000000,15000.000000
4,00044d6e3352,lab_alb,5.741667,30.000000,0.793103,30.000000
5,00044d6e3352,lab_alp,5.741667,46.000000,0.032653,46.000000
6,00044d6e3352,lab_alt,5.741667,10.000000,0.005325,10.000000
7,00044d6e3352,lab_amy,92.105000,44.000000,445.000000,44.000000
8,00044d6e3352,lab_ast,5.741667,12.000000,0.004600,12.000000
9,00044d6e3352,lab_ca,5.741944,1.980000,0.500000,1.980000


In [203]:
summ = pd.pivot_table(labs_sub, 
                      index = 'ENCOUNTER_NUM', 
                      columns = 'variable', 
                      values = 'numeric_value_imputed', 
                      aggfunc = np.sum)

summ.replace(0, np.nan, inplace = True)
summ

variable,alcohol_sciwascore,diabetic_spocglucresult,in_out_acatheter,in_out_atmsincontinent,in_out_siv23and13,in_out_sivnormalsaline,lab_acet,lab_alact,lab_alb,lab_alp,...,skin_abradenscore,vital_sbpdiastolic,vital_sbpsystolic,vital_sfio2,vital_so2saturation,vital_spainintmove,vital_spainintrest,vital_spulse,vital_srespirations,vital_stemperature
ENCOUNTER_NUM,,,,,,,,,,,,,,,,,,,,,
00044d6e3352,9.0,NaN,8500.0,NaN,6250.0,15000.0,NaN,NaN,30.000000,46.000000,...,12.0,68.0,122.0,NaN,97.0,3.0,3.0,80.0,24.0,36.5
00052514c30f,NaN,NaN,NaN,20.0,NaN,NaN,NaN,NaN,0.517241,0.077062,...,16.0,90.0,156.0,NaN,97.0,NaN,NaN,72.0,20.0,35.5
0007e6523e5d,NaN,13.5,11000.0,NaN,NaN,NaN,NaN,NaN,28.000000,0.227702,...,NaN,93.0,173.0,NaN,96.0,NaN,NaN,80.0,20.0,36.3
000b0e7e7ae9,NaN,NaN,NaN,NaN,NaN,18000.0,NaN,NaN,33.000000,44.000000,...,NaN,61.0,105.0,NaN,97.0,NaN,NaN,60.0,20.0,35.9
0013da1c712c,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.000000,0.064000,...,NaN,85.0,134.0,NaN,97.0,6.0,6.0,82.0,18.0,35.6
0014899413c8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,32.000000,0.022204,...,NaN,66.0,110.0,NaN,92.0,NaN,NaN,83.0,18.0,35.9
0016e80ade32,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,32.000000,0.052658,...,16.0,83.0,143.0,NaN,100.0,9.0,9.0,112.0,20.0,37.0
0016fd45df95,NaN,NaN,4000.0,NaN,NaN,2500.0,NaN,NaN,26.000000,206.000000,...,NaN,68.0,104.0,NaN,98.0,NaN,NaN,92.0,18.0,36.5
001ea4a5e978,NaN,11.8,NaN,NaN,NaN,NaN,NaN,NaN,36.000000,332.000000,...,23.0,71.0,111.0,NaN,99.0,NaN,NaN,99.0,20.0,36.6


In [204]:
summ.isnull().mean().sort_values(ascending = False)
# summ.isnull().mean().sort_values(ascending = False).reset_index(drop = False).to_csv('lab_pct_missing.csv')

variable
lab_alact                  0.996163
shift_assess_srpfio2b      0.974338
lab_asa                    0.970022
lab_etoh                   0.969702
lab_apco2                  0.968023
lab_aph                    0.968023
lab_apo2                   0.968023
lab_atco2                  0.968023
vital_sfio2                0.965785
lab_acet                   0.963546
in_out_atmsincontinent     0.950755
shift_assess_scvhrtrate    0.947718
in_out_siv23and13          0.946678
lab_bnps                   0.945799
alcohol_sciwascore         0.938604
lab_crp                    0.922056
lab_bc                     0.910944
lab_sat                    0.865297
lab_fer                    0.857063
diabetic_spocglucresult    0.843952
lab_hba1                   0.820609
lab_b12                    0.810057
in_out_acatheter           0.804301
lab_tpr                    0.803901
shift_assess_srpo2lmin     0.784075
lab_tsh                    0.772164
vital_spainintrest         0.737229
vital_spainintmove 

In [205]:
to_keep = ['lab_alp', 'lab_ast', 'lab_ca', 'lab_ck', 'lab_cl', 'lab_co2', 'lab_amy', 'lab_cr', 'lab_alt', 
           'lab_alb', 'vital_stemperature', 'lab_glur', 'lab_hgb', 'vital_spulse', 'vital_so2saturation', 
           'vital_sbpsystolic', 'vital_sbpdiastolic', 'lab_vtco2', 'lab_vpo2', 'lab_vph', 'lab_vpco2', 
           'lab_vlact', 'lab_urea', 'lab_tbil', 'lab_rptt', 'lab_rpt', 'lab_rinr', 'lab_po4', 'lab_plt', 
           'lab_ph', 'lab_na', 'lab_mg', 'lab_lip', 'vital_srespirations', 'lab_iwbcr', 'lab_k']

summ[to_keep].isnull().mean().sort_values(ascending = False)

variable
lab_ph                 0.060436
lab_lip                0.035175
lab_alt                0.011831
lab_vpo2               0.011592
lab_rptt               0.011352
lab_amy                0.010313
lab_ast                0.007035
lab_alp                0.006955
lab_rinr               0.006955
lab_rpt                0.005436
lab_tbil               0.003757
lab_vpco2              0.003438
lab_ck                 0.003278
lab_glur               0.001999
lab_vtco2              0.001519
lab_vlact              0.001439
lab_urea               0.001199
lab_vph                0.001119
lab_mg                 0.000640
lab_po4                0.000560
lab_alb                0.000320
lab_ca                 0.000320
lab_cr                 0.000240
lab_plt                0.000160
lab_cl                 0.000080
lab_co2                0.000080
lab_k                  0.000080
lab_hgb                0.000080
lab_iwbcr              0.000080
vital_spulse           0.000000
lab_na                 0.000000

In [206]:
summ = summ[to_keep].reset_index(drop = False)
summ = summ.merge(labs_numtypes, 
                  how = 'left', 
                  left_on = 'ENCOUNTER_NUM', 
                  right_index = True).merge(labs_numorders, 
                                            how = 'left', 
                                            left_on = 'ENCOUNTER_NUM', 
                                            right_index = True)

# summ.to_csv('last_lab_measurement.csv', index = False)
summ

,ENCOUNTER_NUM,lab_alp,lab_ast,lab_ca,lab_ck,lab_cl,lab_co2,lab_amy,lab_cr,lab_alt,...,lab_plt,lab_ph,lab_na,lab_mg,lab_lip,vital_srespirations,lab_iwbcr,lab_k,labs_numtypes,labs_num
0,00044d6e3352,46.000000,12.000000,1.980000,118.000000,105.000000,20.000000,44.000000,138.000000,10.000000,...,152.000000,0.142857,133.000000,0.850000,105.000000,24.0,9.470000,4.900000,35.0,157.0
1,00052514c30f,0.077062,0.027599,0.456140,0.003384,98.000000,30.000000,0.038947,93.000000,0.017305,...,273.000000,6.000000,133.000000,0.434783,0.018025,20.0,10.400000,4.400000,21.0,315.0
2,0007e6523e5d,0.227702,0.090591,2.220000,0.012157,107.000000,26.000000,0.014837,154.000000,0.036829,...,266.000000,NaN,140.000000,0.750000,0.018025,20.0,9.580000,5.200000,27.0,123.0
3,000b0e7e7ae9,44.000000,181.000000,1.860000,135.000000,107.000000,24.000000,0.013910,93.000000,111.000000,...,69.000000,0.285714,135.000000,0.590000,0.007965,20.0,1.660000,3.700000,27.0,179.0
4,0013da1c712c,0.064000,0.010222,0.622807,0.003663,104.000000,26.000000,0.038947,87.000000,0.007987,...,217.000000,0.142857,139.000000,0.565217,NaN,18.0,5.020000,4.000000,21.0,93.0
5,0014899413c8,0.022204,0.247368,2.190000,258.000000,102.000000,26.000000,0.038947,84.000000,0.042597,...,257.000000,5.000000,137.000000,0.870000,NaN,18.0,8.030000,3.700000,22.0,106.0
6,0016e80ade32,0.052658,0.749883,2.240000,0.851304,108.000000,27.000000,0.186043,99.000000,0.454209,...,434.000000,0.285714,142.000000,0.700000,0.028505,20.0,11.220000,4.200000,23.0,255.0
7,0016fd45df95,206.000000,13.000000,0.517544,0.000279,101.000000,26.000000,0.038947,153.000000,7.000000,...,208.000000,0.571429,137.000000,0.782609,8.000000,18.0,5.270000,4.400000,31.0,884.0
8,001ea4a5e978,332.000000,21.000000,0.649123,0.003384,95.000000,29.000000,0.029906,74.000000,27.000000,...,682.000000,0.285714,132.000000,0.376812,0.016768,20.0,10.050000,4.600000,28.0,278.0
9,00254e92344d,0.035266,0.008177,2.300000,31.000000,106.000000,25.000000,0.038947,74.000000,0.005325,...,332.000000,0.285714,141.000000,0.750000,0.018025,18.0,12.430000,5.100000,27.0,172.0


In [207]:
summ.isnull().sum()

ENCOUNTER_NUM            0
lab_alp                 87
lab_ast                 88
lab_ca                   4
lab_ck                  41
lab_cl                   1
lab_co2                  1
lab_amy                129
lab_cr                   3
lab_alt                148
lab_alb                  4
vital_stemperature       0
lab_glur                25
lab_hgb                  1
vital_spulse             0
vital_so2saturation      0
vital_sbpsystolic        0
vital_sbpdiastolic       0
lab_vtco2               19
lab_vpo2               145
lab_vph                 14
lab_vpco2               43
lab_vlact               18
lab_urea                15
lab_tbil                47
lab_rptt               142
lab_rpt                 68
lab_rinr                87
lab_po4                  7
lab_plt                  2
lab_ph                 756
lab_na                   0
lab_mg                   8
lab_lip                440
vital_srespirations      0
lab_iwbcr                1
lab_k                    1
l

In [219]:
summ[summ.labs_numtypes.isnull()]

,ENCOUNTER_NUM,lab_alp,lab_ast,lab_ca,lab_ck,lab_cl,lab_co2,lab_amy,lab_cr,lab_alt,...,lab_plt,lab_ph,lab_na,lab_mg,lab_lip,vital_srespirations,lab_iwbcr,lab_k,labs_numtypes,labs_num
2948,3b9e027e6115,0.077062,0.027599,0.631579,0.003384,0.454545,0.434783,0.038947,0.058252,0.017305,...,0.262579,0.857143,0.62069,0.173913,0.018025,0.230769,0.323451,0.5,NaN,NaN


In [221]:
labs_all[labs_all.ENCOUNTER_NUM == '3b9e027e6115']

,ENCOUNTER_NUM,variable,time,numeric_value,numeric_value_prev,numeric_value_imputed
729865,3b9e027e6115,lab_alb,0.0,0.827586,NaN,0.827586
729866,3b9e027e6115,lab_alp,0.0,0.077062,NaN,0.077062
729867,3b9e027e6115,lab_alt,0.0,0.017305,NaN,0.017305
729868,3b9e027e6115,lab_amy,0.0,0.038947,NaN,0.038947
729869,3b9e027e6115,lab_ast,0.0,0.027599,NaN,0.027599
729870,3b9e027e6115,lab_ca,0.0,0.631579,NaN,0.631579
729871,3b9e027e6115,lab_ck,0.0,0.003384,NaN,0.003384
729872,3b9e027e6115,lab_cl,0.0,0.454545,NaN,0.454545
729873,3b9e027e6115,lab_co2,0.0,0.434783,NaN,0.434783
729874,3b9e027e6115,lab_cr,0.0,0.058252,NaN,0.058252


In [240]:
final2 = final.merge(summ, on = 'ENCOUNTER_NUM', how = 'left')

final2.isnull().sum(axis = 0)

ENCOUNTER_NUM            0
PATIENT_DK               0
FROM_SERVICE             0
ADT_DISCHARGE          837
MRP_DIAGNOSIS          190
age                      0
gim_to_outcome           0
pre_gim_icu              0
post_gim_icu             0
OUTCOME_TYPE             0
marital                  0
language                 0
religion                 0
gender                   0
readmit_30d              0
med_08                  42
med_10                  42
med_12                  42
med_20                  42
med_24                  42
med_28                  42
med_40                  42
med_48                  42
med_56                  42
med_68                  42
med_72                  42
med_80                  42
med_86                  42
med_88                  42
med_92                  42
                      ... 
lab_alt                148
lab_alb                  4
vital_stemperature       0
lab_glur                25
lab_hgb                  1
vital_spulse             0
v

In [241]:
final2.labs_numtypes = final2.labs_numtypes.fillna(0)
final2.labs_num = final2.labs_num.fillna(0)

In [242]:
final2[final2.lab_k.isnull()]

,ENCOUNTER_NUM,PATIENT_DK,FROM_SERVICE,ADT_DISCHARGE,MRP_DIAGNOSIS,age,gim_to_outcome,pre_gim_icu,post_gim_icu,OUTCOME_TYPE,...,lab_plt,lab_ph,lab_na,lab_mg,lab_lip,vital_srespirations,lab_iwbcr,lab_k,labs_numtypes,labs_num
11319,17d97ecbebad,129235,TME GENERAL MEDICINE,Home Without Services,Gastrointestinal haemorrhage NOS,62.482324,1.657269,0,0,5,...,296.0,0.285714,0.517241,0.434783,0.018025,18.0,7.74,NaN,12.0,62.0


In [243]:
labs_all[(labs_all.ENCOUNTER_NUM == '17d97ecbebad') & (labs_all.variable == 'lab_k')]

,ENCOUNTER_NUM,variable,time,numeric_value,numeric_value_prev,numeric_value_imputed
289882,17d97ecbebad,lab_k,0.0,NaN,NaN,NaN


In [244]:
labsb[(labsb.ENCOUNTER_NUM == '17d97ecbebad') & (labsb.variable == 'lab_k')]

,ENCOUNTER_NUM,variable,numeric_value,time
324966,17d97ecbebad,lab_k,0.0,0


# Clinical Orders

In [235]:
# Clinical orders
test = pd.read_csv('raw_data/outcsv_test_clinical_orders.csv')
train = pd.read_csv('raw_data/outcsv_train_train_clinical_orders.csv')
val = pd.read_csv('raw_data/outcsv_valid_valid_clinical_orders.csv')

co = pd.concat([train, test, val], axis = 0).reset_index(drop = True)
co.head()

# relevant_cos = ['consult_addiction', 'consult_chaplain', 'consult_dietitian', 'consult_occupational', 
#                 'consult_pharmacist', 'consult_physio', 'consult_social', 'consult_speech', 'consult_wound']

co_sub = co[co.order_name.str.startswith('consult_')].reset_index(drop = True)

In [236]:
summ_co = pd.pivot_table(co_sub, 
                         index = 'ENCOUNTER_NUM', 
                         columns = 'order_name', 
                         values = 'value', 
                         aggfunc = np.sum)

summ_co = summ_co.clip(upper = 1).reset_index(drop = False)
summ_co

order_name,ENCOUNTER_NUM,consult_acute,consult_addiction,consult_chaplain,consult_chiropodist,consult_dietitian,consult_gastroenterology,consult_general,consult_geriatric,consult_occupational,...,consult_physio,consult_physiotherapist,consult_physiotherapy,consult_psychiatry,consult_research,consult_respiratory,consult_social,consult_speech,consult_stroke,consult_wound
0,00044d6e3352,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,00052514c30f,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,...,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,0014899413c8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,0016e80ade32,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,0016fd45df95,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,...,NaN,1.0,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN
5,001cdc675da1,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
6,001ea4a5e978,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,0020e8b40d41,NaN,NaN,NaN,NaN,1.0,NaN,1.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN
8,002a0537de20,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,002abc4e987f,NaN,NaN,NaN,NaN,1.0,NaN,1.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [109]:
cardiac_cos = ['telemetry', 'cardio_ecg', 'cardio_vascularlab', 'cardio_echo', 
               'cardio_holter', 'cardio_peripheralvascular']

cardiac = co[co.order_name.isin(cardiac_cos)].reset_index(drop = True)

In [148]:
cardiac_numorders = cardiac.groupby(['ENCOUNTER_NUM'])['order_name'].count().rename('cardiac_numorders')
cardiac_numtypes = cardiac.groupby(['ENCOUNTER_NUM'])['order_name'].nunique().rename('cardiac_numtypes')

In [159]:
img_ang0090 = co.groupby(['ENCOUNTER_NUM']).apply(lambda x: sum(x.order_name.isin(['img_ang0090']))).rename('img_ang0090')

In [151]:
resp_cos = ['resp_oxygen', 'resp_pulmonaryfunctiontest', 'resp_bipapcpap', 
            'resp_respiratoryintervention', 'resp_chesttube']

resp = co[co.order_name.isin(resp_cos)].reset_index(drop = True)

resp_numorders = resp.groupby(['ENCOUNTER_NUM'])['order_name'].count().rename('resp_numorders')
resp_numtypes = resp.groupby(['ENCOUNTER_NUM'])['order_name'].nunique().rename('resp_numtypes')

In [160]:
resp_ventilator = co.groupby(['ENCOUNTER_NUM']).apply(lambda x: sum(x.order_name.isin(['resp_ventilator']))).rename('resp_ventilator')

In [153]:
lines_cos = ['img_intpcin', 'img_intpcsl']

lines = co[co.order_name.isin(lines_cos)].reset_index(drop = True)

lines_numorders = lines.groupby(['ENCOUNTER_NUM'])['order_name'].count().rename('lines_numorders')
lines_numtypes = lines.groupby(['ENCOUNTER_NUM'])['order_name'].nunique().rename('lines_numtypes')

In [154]:
transfusion_cos = ['trans_infusefrozenplasma', 'trans_transfusepackedredbloodcells', 'trans_transfuseplatelets', 
                   'trans_infusealbumin25', 'trans_transfusionother', 'trans_infuseivimmuneglobulin', 
                   'trans_prothrombincomplexconcentratepcc']

transfusion = co[co.order_name.isin(transfusion_cos)].reset_index(drop = True)

transfusion_numorders = transfusion.groupby(['ENCOUNTER_NUM'])['order_name'].count().rename('transfusion_numorders')
transfusion_numtypes = transfusion.groupby(['ENCOUNTER_NUM'])['order_name'].nunique().rename('transfusion_numtypes')

In [161]:
trans_transfusepackedredbloodcells = co.groupby(['ENCOUNTER_NUM']).apply(lambda x: sum(x.order_name.isin(['trans_transfusepackedredbloodcells']))).rename('trans_transfusepackedredbloodcells')

In [156]:
radio_cos = list(set(co.order_name[co.order_name.str.startswith('img_')]) - set(['img_ang0090', 'img_intpcin', 'img_intpcsl']))

radio = co[co.order_name.isin(radio_cos)].reset_index(drop = True)

radio_numorders = radio.groupby(['ENCOUNTER_NUM'])['order_name'].count().rename('radio_numorders')
radio_numtypes = radio.groupby(['ENCOUNTER_NUM'])['order_name'].nunique().rename('radio_numtypes')

In [164]:
to_add = pd.concat([cardiac_numorders, cardiac_numtypes, img_ang0090, 
                    resp_numorders, resp_numtypes, resp_ventilator, 
                    transfusion_numorders, transfusion_numtypes, trans_transfusepackedredbloodcells,
                    radio_numorders, radio_numtypes, lines_numorders, lines_numtypes], axis = 1)

to_add = to_add.fillna(0)
to_add

C:\Users\alexw\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  after removing the cwd from sys.path.


,cardiac_numorders,cardiac_numtypes,img_ang0090,resp_numorders,resp_numtypes,resp_ventilator,transfusion_numorders,transfusion_numtypes,trans_transfusepackedredbloodcells,radio_numorders,radio_numtypes,lines_numorders,lines_numtypes
00044d6e3352,1.0,1.0,0,0.0,0.0,0,0.0,0.0,0,2.0,2.0,0.0,0.0
00052514c30f,0.0,0.0,0,0.0,0.0,0,0.0,0.0,0,0.0,0.0,0.0,0.0
0013da1c712c,0.0,0.0,0,0.0,0.0,0,0.0,0.0,0,3.0,3.0,0.0,0.0
0014899413c8,0.0,0.0,0,0.0,0.0,0,0.0,0.0,0,1.0,1.0,0.0,0.0
0016e80ade32,2.0,2.0,0,0.0,0.0,0,0.0,0.0,0,0.0,0.0,0.0,0.0
0016fd45df95,3.0,3.0,0,1.0,1.0,0,0.0,0.0,0,2.0,2.0,0.0,0.0
001cdc675da1,0.0,0.0,0,0.0,0.0,0,2.0,1.0,2,5.0,3.0,2.0,1.0
001ea4a5e978,0.0,0.0,0,0.0,0.0,0,0.0,0.0,0,3.0,1.0,0.0,0.0
001f25e866fb,0.0,0.0,0,0.0,0.0,0,0.0,0.0,0,1.0,1.0,0.0,0.0
0020e8b40d41,1.0,1.0,0,0.0,0.0,0,0.0,0.0,0,2.0,1.0,0.0,0.0


In [237]:
summ_co = summ_co.merge(to_add, how = 'left', left_on = 'ENCOUNTER_NUM', right_index = True)

In [238]:
summ_co

,ENCOUNTER_NUM,consult_acute,consult_addiction,consult_chaplain,consult_chiropodist,consult_dietitian,consult_gastroenterology,consult_general,consult_geriatric,consult_occupational,...,resp_numorders,resp_numtypes,resp_ventilator,transfusion_numorders,transfusion_numtypes,trans_transfusepackedredbloodcells,radio_numorders,radio_numtypes,lines_numorders,lines_numtypes
0,00044d6e3352,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,0.0,0,0.0,0.0,0,2.0,2.0,0.0,0.0
1,00052514c30f,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,...,0.0,0.0,0,0.0,0.0,0,0.0,0.0,0.0,0.0
2,0014899413c8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,...,0.0,0.0,0,0.0,0.0,0,1.0,1.0,0.0,0.0
3,0016e80ade32,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,...,0.0,0.0,0,0.0,0.0,0,0.0,0.0,0.0,0.0
4,0016fd45df95,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,...,1.0,1.0,0,0.0,0.0,0,2.0,2.0,0.0,0.0
5,001cdc675da1,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,...,0.0,0.0,0,2.0,1.0,2,5.0,3.0,2.0,1.0
6,001ea4a5e978,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,...,0.0,0.0,0,0.0,0.0,0,3.0,1.0,0.0,0.0
7,0020e8b40d41,NaN,NaN,NaN,NaN,1.0,NaN,1.0,NaN,NaN,...,0.0,0.0,0,0.0,0.0,0,2.0,1.0,0.0,0.0
8,002a0537de20,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,...,0.0,0.0,0,0.0,0.0,0,0.0,0.0,0.0,0.0
9,002abc4e987f,NaN,NaN,NaN,NaN,1.0,NaN,1.0,NaN,NaN,...,0.0,0.0,0,0.0,0.0,0,1.0,1.0,0.0,0.0


In [245]:
final3 = final2.merge(summ_co, on = 'ENCOUNTER_NUM', how = 'left')

for c in summ_co.columns.values:
    final3[c] = final3[c].fillna(0)

In [246]:
final3.head()

,ENCOUNTER_NUM,PATIENT_DK,FROM_SERVICE,ADT_DISCHARGE,MRP_DIAGNOSIS,age,gim_to_outcome,pre_gim_icu,post_gim_icu,OUTCOME_TYPE,...,resp_numorders,resp_numtypes,resp_ventilator,transfusion_numorders,transfusion_numtypes,trans_transfusepackedredbloodcells,radio_numorders,radio_numtypes,lines_numorders,lines_numtypes
0,0013da1c712c,668421,TMC GENERAL MEDICINE,Home Without Services,Cutan abscess furuncle & carbuncle trunk,41.869648,2.674120,0,0,5,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0014899413c8,69698,TMA GENERAL MEDICINE,Home Without Services,NaN,89.221340,3.710741,1,0,5,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0
2,0016fd45df95,838638,TMC GENERAL MEDICINE,Long Term Care/Nursing Home/Detox,Congestive heart failure,91.860508,24.363495,0,0,5,...,1.0,1.0,0.0,0.0,0.0,0.0,2.0,2.0,0.0,0.0
3,0063dea6173d,127708,TMC GENERAL MEDICINE,Home Without Services,NaN,61.269224,4.557257,0,0,5,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,00b661c2b87e,850511,TME GENERAL MEDICINE,Home Without Services,NaN,71.207212,1.820116,0,0,5,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [247]:
final3.shape

(12509, 103)

In [248]:
final3.isnull().sum(axis = 0)

ENCOUNTER_NUM                           0
PATIENT_DK                              0
FROM_SERVICE                            0
ADT_DISCHARGE                         837
MRP_DIAGNOSIS                         190
age                                     0
gim_to_outcome                          0
pre_gim_icu                             0
post_gim_icu                            0
OUTCOME_TYPE                            0
marital                                 0
language                                0
religion                                0
gender                                  0
readmit_30d                             0
med_08                                 42
med_10                                 42
med_12                                 42
med_20                                 42
med_24                                 42
med_28                                 42
med_40                                 42
med_48                                 42
med_56                            

In [249]:
final3.to_csv('semi_final3.csv', index = False)
final3.to_csv('semi_final3.txt', index = False) # Some encounter numbers may not be displayed properly in Excel (e.g. leading 0)

# Add Age Groups

In [265]:
final3.loc[:, 'age_group'] = pd.cut(final3.age, bins = [0, 50, 55, 60, 65, 70, 75, 80, 85, 90, 95, 100, 150], right = True)

In [266]:
final3.age_group.value_counts()

(0, 50]       2635
(80, 85]      1386
(65, 70]      1239
(75, 80]      1232
(70, 75]      1150
(85, 90]      1112
(60, 65]      1070
(55, 60]      1034
(50, 55]       901
(90, 95]       564
(95, 100]      164
(100, 150]      22
Name: age, dtype: int64

In [268]:
final3.to_csv('semi_final3.txt', index = False)

# Add Diagnosis Categories

In [302]:
dt = pd.read_csv('semi_final4.csv').drop(columns = ['Unnamed: 0'])

In [303]:
dt.shape

(12509, 106)

In [291]:
disease_lu = pd.read_csv('Disease.csv')
disease_lu.MRP_grouped = disease_lu.MRP_grouped.str.lower()
disease_lu

,MRP_DIAGNOSIS,MRP_grouped
0,2nd dgr perineal laceration del/deliv,other
1,Abn findings on diagnostic imaging lung,other
2,Abnormal results kidney function studies,other
3,Abnormal results of liver fn studies,other
4,Abnormal uterine & vaginal bleeding NOS,other
5,Abnormal weight loss,other
6,Abscess of liver,other
7,Abscess of lung with pneumonia,other
8,Abscess of lung without pneumonia,other
9,Abscess of prostate,other


In [292]:
dt = tst.merge(disease_lu, on = 'MRP_DIAGNOSIS', how = 'left')
dt.MRP_grouped.isnull().sum()

190

In [293]:
dt[dt.MRP_grouped.isnull()].MRP_DIAGNOSIS.value_counts()

Series([], Name: MRP_DIAGNOSIS, dtype: int64)

In [295]:
dt.MRP_grouped = dt.MRP_grouped.replace(np.nan, 'Unknown')

In [296]:
dt.MRP_grouped.value_counts()

other                      6654
pneumonia                   682
general symptoms            583
copd                        504
uti                         457
heart failure               440
delirium                    369
diabetes                    334
electrolyte abnormality     267
cellulitis                  263
kidney injury               244
sepsis                      214
stroke                      196
Unknown                     190
seizure                     184
cirrhosis/hepatitis         157
gi symptom                  146
poisoning                   145
aspiration pneumonia        123
syncope                     116
arrhythmia                   77
severe enteritis             74
non-severe enteritis         56
gi bleed                     37
palliative care               3
Name: MRP_grouped, dtype: int64

In [299]:
dt.head()

,ENCOUNTER_NUM,PATIENT_DK,FROM_SERVICE,ADT_DISCHARGE,MRP_DIAGNOSIS,age,gim_to_outcome,pre_gim_icu,post_gim_icu,OUTCOME_TYPE,...,transfusion_numtypes,trans_transfusepackedredbloodcells,radio_numorders,radio_numtypes,lines_numorders,lines_numtypes,age_group,sepsis,resp,MRP_grouped
0,00044d6e3352,194507,TMA GENERAL MEDICINE,Home Without Services,Alcohol induced acute pancreatitis,78.543522,4.124803,0,0,5,...,0,0,2,2,0,0,"(75, 80]",0,0,other
1,00052514c30f,87670,TME GENERAL MEDICINE,Long Term Care/Nursing Home/Detox,Pneumonia unspecified,90.837623,13.624688,0,0,5,...,0,0,0,0,0,0,"(90, 95]",0,0,pneumonia
2,0007e6523e5d,389169,INTENSIVE CARE MEDICAL,NaN,Type 2 DM with ketoacidosis,48.919463,3.543426,1,0,5,...,0,0,0,0,0,0,"(0, 50]",0,0,diabetes
3,000b0e7e7ae9,412413,TMC GENERAL MEDICINE,Home Without Services,Fever unspecified,28.105280,4.969271,0,0,5,...,0,0,0,0,0,0,"(0, 50]",1,0,general symptoms
4,0013da1c712c,668421,TMC GENERAL MEDICINE,Home Without Services,Cutan abscess furuncle & carbuncle trunk,41.869648,2.674120,0,0,5,...,0,0,0,0,0,0,"(0, 50]",0,0,other


In [298]:
dt.to_csv('semi_final4.txt', index = False)